# 14.TRPO （Trust Region Policy Optimization）：信任区域策略优化
> 即使 **Actor-Critic** 在一定程度上能够降低训练过程中的方差，在实际应用过程中仍与传统策略梯度算法一样存在着 **训练不稳定** 的情况。根本原因在于策略更新过大：网络参数对样本传递的环境信息 **“过度拟合”**，导致训练时全局策略变差，造成训练的不稳定。特别是当面临情况复杂，策略网络是深度模型时，沿着策略梯度更新参数，步长太长，在一段时间内陷入对部分样本的过分信任，会导致全局策略突然显著变差。

> **TRPO（Trust Region Policy Optimization）算法** 最初由 John Schulman 等人于2015年在 [Trust Region Policy Optimization](https://arxiv.org/abs/1502.05477) 中提出。
> 论文引入了 **信任区域（trust region）和KL散度约束** 的概念。在 **信任区域** 上更新策略时能够得到某种策略性能的安全性保证，是其主要思想。**TRPO** 描述了一种优化策略的迭代过程：在理论上能够保证策略学习的 **性能单调性**，并在实际应用中取得了比策略梯度算法更好的效果。

### 策略目标：
> 由于无论策略为什么，采样一条轨迹的初始状态$s_0$的分布$p(s_0)$固定和策略$\pi$无关，也就是说:
$$\begin{aligned}
J(\theta) & =\mathbb{E}_{\pi}[V^{\pi_\theta}(s_0)] =\sum_{s_0}p(s_0)V^{\pi_\theta}(s_0) \\
 & =\mathbb{E}_{\pi_{\theta^{\prime}}}[V^{\pi_\theta}(s_0)] \\
 & =\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^\infty\gamma^tV^{\pi_\theta}(s_t)-\sum_{t=1}^\infty\gamma^tV^{\pi_\theta}(s_t)\right] \\
 & =-\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^{\infty}\gamma^{t}\left(\gamma V^{\pi_{\theta}}(s_{t+1})-V^{\pi_{\theta}}(s_{t})\right)\right]
\end{aligned}$$

> 基于以上等式，可以推导新旧策略的目标函数之间的差距：
$$\begin{aligned}
J(\theta^{\prime})-J(\theta) & =\mathbb{E}_{\pi}\left[V^{\pi_{\theta^{\prime}}}(s_0)\right]-\mathbb{E}_{\pi}\left[V^{\pi_\theta}(s_0)\right] \\
 & =\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^\infty\gamma^tr(s_t,a_t)\right]+\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^\infty\gamma^t\left(\gamma V^{\pi_\theta}(s_{t+1})-V^{\pi_\theta}(s_t)\right)\right] \\
 & =\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^\infty\gamma^t\left[r(s_t,a_t)+\gamma V^{\pi_\theta}(s_{t+1})-V^{\pi_\theta}(s_t)\right]\right]
\end{aligned}$$


> 可见上述公式存在 **时序差分残差** 形式，以下转换成**优势函数A**：
